Please enter your email to access the NCBI Entrez records (useful for getting the species name to which each protein belongs to)

In [45]:
entrez_email = 'fp1n17@soton.ac.uk'

In [46]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, sys, os, collections, itertools, holoviews as hv, multiprocessing
from tqdm import tqdm
from datetime import datetime
from collections import Counter

from sklearn.model_selection import StratifiedKFold, GridSearchCV,train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score, roc_curve
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pickle
from Bio import SeqIO
from Bio import Entrez

In [47]:
'''GENERATE INDICES FOR CONSTRUCTING TRAIN-TEST SET PAIRS FOR EACH SPECIES IN LOBOV'''

'GENERATE INDICES FOR CONSTRUCTING TRAIN-TEST SET PAIRS FOR EACH SPECIES IN LOBOV'

In [48]:
'''setting data Paths'''
prots_path='./Supplementary_Table_2.xlsx'
trigrams_path='./protVec_100d_3grams.csv'

tut_path = 'tut-3-lobov'
d_path = os.path.join(tut_path,'bpad200_lobov_splits_indic')
if os.path.exists(d_path) == False :
    os.makedirs(d_path)

In [49]:
'''get species of each protein from NCBI'''

prots_data = pd.read_excel(prots_path
                             ,usecols = ['Id'],index_col=None
                            ,skiprows=[0,1])


prots_data

,Id
0,1DI0_A
1,1DYQ
2,2Y1V_C
3,A46405
4,AAA20874.1
...,...
395,YP_222757.1
396,YP_695964.1
397,YP_696046.1
398,YP_763859.1


In [ ]:
'''Cell has large time complexity (around 5 mins to run)'''

#load the ijms-18-00312-s001.fa file to get the sequence of each protein
#TODO: 'ijms-18-00312-s001.fa' does not have sequences for all proteins...!!!!
#and also has different accession names than NCBI protein db (extra/missing '.' , '_' and other characters)

#get the sequences from NCBI 
#check 'Entrez Guidelines' http://biopython.org/DIST/docs/tutorial/Tutorial.html#chapter%3Aentrez
#code ref: https://www.ncbi.nlm.nih.gov/dbvar/content/tools/entrez/
species_names=[]
Entrez.email  = entrez_email
db = 'Protein'

for i,acc in enumerate(prots_data.loc[:,'Id'].values.tolist()):
    if i%50==0:
        print('Protein species name collected: ',i,'   Current protein collected: ',acc,'\n')
    #testing
    #print(species_names,'\n',acc,'\n')
    try:
        handle = Entrez.efetch(db="protein", id=acc, rettype="gb", retmode="text")
    except Exception:
        if '_' not in acc:
            acc = acc+'_A' 
            try:
                handle = Entrez.efetch(db="protein", id=acc, rettype="gb", retmode="text")
            except Exception:
                if '.' not in acc:
                    acc = acc[:-2]+'.1'
                    handle = Entrez.efetch(db="protein", id=acc, rettype="gb", retmode="text")
    
    record = SeqIO.read(handle, "genbank")
    handle.close()
    
    species_names.append(str(record.annotations['organism']))


Protein species name collected:  0    Current protein collected:  1DI0_A 



C:\Users\Stelios\anaconda3\envs\Frixos\lib\site-packages\Bio\GenBank\__init__.py:1139: BiopythonParserWarning: Dropping bond qualifier in feature location
  warnings.warn(


In [ ]:
#species_names

In [ ]:
#create proteins dataset and store in the desired format for use later on

prots_data = pd.concat([prots_data.iloc[:,0],pd.DataFrame(species_names,columns = ['Species'])],axis=1)
#prots_data.columns=cols_
prots_data.rename({'Id':'Accession'},inplace=True,axis=1)
prots_data.to_csv(tut_path+'\\bpad200_species.csv')


In [ ]:
'''function handling generation of test set indices for LOBOV'''

def store_lobov_splits(dataset_df,acc_spec_df,thresh):
    '''function that takes full @dataset_df of proteins and separately accessions_species df @acc_spec_df 
    for the same proteins as arguments, returns the train test splits for outer CV, 
    where each test-set contains proteins from the same bacterial species 
    (this was only done where we have more than @thresh proteins overall for that species)'''
    '''this fn is found useful in cases where we already have the original dataset
    but without the species, which we receive at some later point in the proj'''
    '''@acc_spec_df should have species as index when passed to this df'''
    
    species_col = []
    
    if set(dataset['Accession'].values.tolist())!=set(acc_spec['Accession'].values.tolist()):
        # apply acc.split('.')[0] to have equal set(dataset['Accession']) and set(acc_spec['Accession'])
        dataset['Accession'].apply(lambda x: x.split('.')[0]   )
        acc_spec['Accession'].apply(lambda x: x.split('.')[0])
    
    #get species for each protein
    for acc in dataset_df['Accession']:
        species_col.append(acc_spec[acc_spec['Accession']==acc]['Species'].values[0]) 
    
    #take each protein's species, keep only the first 2 words in the string specifying the species
    #,remove any double whitespace (of any kind, even tab characters) , remove trailing whitespaces on both sides
    species_col = [' '.join(s.split()[:2]).lstrip().rstrip() for s in species_col]
    
    #where there are ambiguous names, such as with Borrelia burgdorferi, Chlamydophila pneumoniae, we kept the names used by Heinson et. al 2017
    species_col_ = []
    for species in species_col:
        if species=='Borreliella burgdorferi':
            species = 'Borrelia burgdorferi'
        elif species=='Chlamydia pneumoniae':
            species = 'Chlamydophila pneumoniae'
        
        species_col_.append(species)
    
    #testing
    #print(species_col_)
    #input()
    
    #assert len(set(species_col))==42
    
    dataset_df['Species'] = species_col_
    
    count_spec = Counter(species_col_)
    lobov_spec = [s for s in count_spec if count_spec[s] >4 ]
    
    #print((lobov_spec),'\n')
    #input()
    
    assert len(lobov_spec)==25
    
    s_dir = os.path.join(tut_path,'bpad200_lobov_splits_indic')
    if os.path.exists('bpad200_lobov_splits_indic') == False : 
        os.mkdir('bpad200_lobov_splits_indic')
    
    
    for spec in lobov_spec:
        test_indic = dataset_df[dataset_df['Species']==spec].index.tolist()
        train_indic = [i for i in range(400) if i not in test_indic]

        df = pd.DataFrame([test_indic,train_indic]).T
        df.columns = ['Test','Train']
        
        #testing
        #input()
        
        #store training/test indic in file for each species
        df.to_csv(s_dir+'\\'+spec+'.csv',index=None)
     
    
    print('\n')
    print([(s,count_spec[s]) for s in count_spec if count_spec[s] >4 ])

In [ ]:
'''load initial proteins dataset'''

prots_path = './BPAD200.csv'

'''load proteins'''
#load bpad200 proteins
dataset = pd.read_csv(prots_path
                      ,index_col=None,usecols=['Accession','Sequence','Label'])
#dataset.reset_index(inplace=True,drop=False)

# handling empty lines between protein records dataset given
if len(dataset)>400:
    dataset= dataset[1::2]
    dataset.reset_index(drop=False,inplace=True)

# testing
assert len(dataset)==400

'''load accessions-species '''
acc_spec = pd.read_csv(tut_path+'\\bpad200_species.csv',index_col=[0])

In [ ]:
'''code-block to get the files with the indices of test set for each species'''
threshold=4
#if os.path.exists('./bpad200_lobov_splits_indic.csv')==False:

acc_spec.reset_index(inplace=True,drop=False)
#acc_spec
store_lobov_splits(dataset,acc_spec,threshold)

In [ ]:
'''UTILITY CLASS'''


class Load_data_model_selection():
    def __init__(self,prot_path=None,prot_cols=None,prot_ind=None):
        #print('\nRefactor function below to work on class instances variables and convert clas to inherit from a more generic load_data class, which class will be parent for all load_data classes for all stages in proj pipeline\n')
        
        self.prot_cols = prot_cols
        self.prot_ind = prot_ind
        self.data_df={}
        

    def get_all_data(self):
        return self.data_df

        
    def populate_data_df(self,all_paths): 
        if isinstance(all_paths,list)==False:
            all_paths = [all_paths]
        
        for data_path in all_paths:
            ##debugging
            ##print('\n',split_fully(data_path),'\n',data_path,'\n')
            self.data_df.setdefault('preProcessing_1',[]).append( split_fully(data_path)[-3] )
            self.data_df.setdefault('preProcessing_2',[]).append( split_fully(data_path)[-2] )
            
            self.data_df.setdefault('name',[]).append( os.path.basename(data_path).split('.')[0] )
            self.data_df.setdefault('dataset',[]).append( self.load_prot_vecs(prot_path = data_path) )        

        self.data_df = pd.DataFrame(self.data_df)
        
        
    def load_prot_vecs(self,prot_path,cols=None,ind=None):
        if ind!=None and cols!=None:
            self.prot_cols = cols
            self.prot_ind=ind
        
        if self.prot_cols=='infer':
            prot_vecs = pd.read_csv(prot_path,
                               header=self.prot_cols,
                               index_col=self.prot_ind,)
                               #skip_blank_lines= True)
        elif isinstance(self.prot_cols,list):
            prot_vecs= pd.read_csv(prot_path,
                               usecols=self.prot_cols,
                               index_col=self.prot_ind,)
        
        
        return prot_vecs
    

def get_dict_key_ind(dict_keys,key_name):
    '''fn that returns the index of the @key_name arg
    in the @keys_list of a dict_ (dict_ present at code block where this fn is called)
    Particularly useful when we want to get value of dict entry by key name, when
    we are iterating over entries of dict_ (not possible otherwise)
    '''
    dict_keys = [k for k in dict_keys]
    ind = dict_keys.index(key_name)
    
    return ind

    '''a note on iterating dictionaries by row (as we conceptually think dfs):
        don't use .items(), it returns (key,value) pair for each key (ie: 4 tuples if we have for keys in dict)
    '''
       
    
def split_fully(path):
    '''split a windows path to all its parts (folders,files)'''
    #ref: https://www.oreilly.com/library/view/python-cookbook/0596001673/ch04s16.html
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts

In [ ]:
'''UTILITY CLASS'''

"""
utility class for classification
-- this class handles just 1 model instance at a time
"""

class Classifier_model():
    def __init__(self,method=None,data_name=None,dataset=None,dataset_path=None,
                 preProcessing_1=None,preProcessing_2=None,split_method=None
                 ,outer_splits=None,inner_splits=None,hparams_model=None):
        RSEED=10
        
        self.best_run_details={}
        np.random.seed(RSEED)
        plt.ioff()
        
        self.method=method
        self.data_name=data_name
        self.dataset=dataset
        self.dataset_path=dataset_path
        self.preProcessing_1=preProcessing_1
        self.preProcessing_2=preProcessing_2
        self.split_method=split_method
        self.outer_splits=outer_splits
        self.inner_splits=inner_splits
        self.hparams_model = hparams_model
        
        self.split_method_path = make_dir([self.dataset_path,self.split_method])
        
        #tut-3-lobov\standardise\PCA\PCs\lobov\LR
        self.classif_method_path = make_dir([self.split_method_path,self.method])
    
      
    def get_run_details(self):
        return self.best_run_details
        
    def get_features(self,dataset):
        return dataset.iloc[:, 2:-1]
        
    def get_labels(self,dataset):
        return dataset.iloc[:, -1]    
    
    
    def outer_cv(self,lobov_data_dir):
        outer_runs_details = {'fold_id':[],'test_auc':[],'best_inner_hparams':[]}
        
        if self.split_method=='lobov':
            print('\n--------------'+self.split_method.upper()+' has started..\n')
            
            #each file in lobov dir corresponds to indices for train data
            #and test data, where test data comprises of proteins from only 1 bact.species
            files_ = os.listdir(lobov_data_dir)
            for file_ in files_:
                file_path = os.path.join(lobov_data_dir,file_)
                species = file_.split('.csv')[0]
                
                best_inner_run_details = self.outer_cv_fold_specie(species,file_path)
                                                
                #saving details for this fold in the outer folds dict
                #inner_runs_details contains best_inner_hparams,test_auc
                outer_runs_details['fold_id'].append(species)
                for key in best_inner_run_details:
                    outer_runs_details[key].append(best_inner_run_details[key])
        
        
        elif self.split_method=='random':
            print('\n'+self.split_method+' OUTER '+str(self.outer_splits)+'-FOLD CV has started..')
            
            cv = StratifiedKFold(n_splits=self.outer_splits,shuffle=True,random_state=10)
        
            for fold_id,(train_indic, test_indic) in enumerate(cv.split(self.get_features(self.dataset),self.get_labels(self.dataset))):
                
                fold_path = make_dir([self.classif_method_path,'fold_'+str(fold_id)])
                best_inner_run_details=self.inner_cv(train_indic,test_indic
                                                    ,fold_id=fold_id,fold_path=fold_path)
        
                
                outer_runs_details['fold_id'].append(fold_id)
                for key in best_inner_run_details:
                    outer_runs_details[key].append(best_inner_run_details[key])
            
        
        #eval outer cv 
        print('\n***Evaluating models\' performance on testing data during C.V.')
        avg_test_auc,best_hparams,test_auc_std = self.eval_outer_cv(outer_runs_details)
        
        ###############print('\n**Average test AUC score: ',avg_test_auc)
        
        #ref=  https://stackoverflow.com/questions/633127/viewing-all-defined-variables/633134
        self.best_run_details = {'preProcessing_1':self.preProcessing_1,
                                 'preProcessing_2':self.preProcessing_2,
                                 'data_name':self.data_name,
                                 'split_method':self.split_method,
                                 'outer_splits':self.outer_splits,
                                 'inner_splits':self.inner_splits,
                                 'method':self.method,
                                 'avg_test_auc':avg_test_auc,
                                 'test_auc_std':test_auc_std,
                                 'best_hparams':best_hparams}
        
        
    def outer_cv_fold_specie(self,species,file_path):
        print('\nGathering data for bacteria ',species)
        fold_path = make_dir([self.classif_method_path,species])
        
        splits_indic = pd.read_csv(file_path,index_col=None)
        
        #use indices from file to split dataset to train and test
        train_indic = list(splits_indic.loc[:,'Train'].values)
        test_indic = (splits_indic.loc[:,'Test'].dropna(axis=0,inplace=False).values).tolist()
        test_indic = [int(j) for j in test_indic]
         
        best_inner_run_details = self.inner_cv(train_indic,test_indic,fold_id=species,fold_path=fold_path)
        
        return best_inner_run_details 
    
        
    def inner_cv(self,train_indic,test_indic,fold_id,fold_path):
        
        best_inner_run_details= self.classifier_model_fold_run(train_indic,test_indic,fold_id,fold_path)
        
        return best_inner_run_details


    

    def classifier_model_fold_run(self,train_indic,test_indic,fold_id,fold_path):
        '''classifier with inner C.V. for hyperparam. tuning
        the basic results are stored and visualized (ROC_AUC score, accuracy)
        '''
        #ref: #ref: https://github.com/WillKoehrsen/Machine-Learning-Projects/blob/master/Random%20Forest%20Tutorial.ipynb
        # reference:  https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html
        # reference: https://www.datacamp.com/community/tutorials/svm-classification-scikit-learn-python
        
        train_features = self.get_features(self.dataset.iloc[train_indic,:])
        train_labels = self.get_labels(self.dataset.iloc[train_indic,:])
        test_features = self.get_features(self.dataset.iloc[test_indic,:])
        test_labels = self.get_labels(self.dataset.iloc[test_indic,:])
        
        #TODO: EDA
        
       
        print('\n\nStarting GridSearch hyper-parameter tuning on '+self.method+' with '+(self.split_method)+' and fold_id='+str(fold_id)+
                ',  10-fold inner C.V. on  '+self.data_name+'  dataset with/from: '+self.preProcessing_1+'-'+self.preProcessing_2+'\n')
        classifier = self.build_classifier()
        
        #cv instance created within GridSearchCV instance during excecution, has parameter
        #shuffle=False => no randomness present => no need to pass randomSeed val.
        #n_jobs=-1 may cause memory error iussues
        #NOTE! : You cannot nest objects with parallel computing (n_jobs different than 1).
        n_jobs=multiprocessing.cpu_count()
        grid_search = GridSearchCV(estimator = classifier, param_grid=self.hparams_model,
                                   scoring='roc_auc',n_jobs = -1, #computing_server has 6 cores, 12 threads  
                        cv = self.inner_splits, refit=True, 
                        verbose = 1)#,pre_dispatch=2*n_jobs)
        
        grid_search.fit(train_features,train_labels)
        
        print('\nFinished hyperparameter tuning of '+self.method+' on dataset\n')
        best_inner_hparams = grid_search.best_params_
        best_model = grid_search.best_estimator_
        
        print('\nMaking predictions with the best model..\n')
        train_predictions = best_model.predict(train_features)
        #train_svm_probs = float64 array, (shape=#train_samples x 1), col2='probab to be BPA' (label=1='the greater label/positive class'), col1 = '..to be nonBPA'
        train_probs = best_model.predict_proba(train_features)[:, 1]
        #test_predictions:: array
        test_predictions = best_model.predict(test_features)
        test_probs = best_model.predict_proba(test_features)[:, 1]
        
        #save_predictions_classifier(self,'train',train_indic,train_predictions,train_probs,fold_path)
        #save_predictions_classifier(self,'test',test_indic,test_predictions,test_probs,fold_path)
        
        print('\nEvaluating basic metrics on model performance\n')
        test_auc = self.visualise_evaluate(test_labels,test_predictions, test_probs,fold_path)

        #we don't re-use the c.v. models for final model deployment => no need to save model below
        #pickle.dump(best_model,open(fold_path+'\\best_model.pkl','wb'))
        
        #plot feature importance and save it
        if self.method=='XGBoost':
            #TODO: make chart bigger, have to pass ax object here ..
            xgb.plot_importance(best_model)
           # plt.savefig(fold_path+'\\feature_importance.png')
        elif self.method=='RF':
            fi_model = pd.DataFrame({'feature': self.get_features(self.dataset),
                   'importance': best_model.feature_importances_}).\
                    sort_values('importance', ascending = False)
            #fi_model.to_csv(fold_path+'\\feature_importance.csv')


        return {'test_auc':test_auc, 'best_inner_hparams':best_inner_hparams}
    
    

    def build_classifier(self):
        '''GridSearchCV with passed hparams overwrites hparams set here in model obj.defn'''
        if self.method=='SVM':
            #if SVM is slow, search for trick with probability=False & predict_proba stackPOverlfow
            classifier = svm.SVC(degree =1, coef0=0.0, probability=True,
                            verbose=False,random_state=10,**self.hparams_model)
        
        elif self.method=='XGBoost':
            classifier=xgb.XGBClassifier(objective='binary:logistic',eval_metric='auc',
          nthread=1, colsample_bytree=1, subsample=1,colsample_bylevel=0.5,
              nrounds=1,random_state=10,**self.hparams_model)
            
        elif self.method=='LR':
            classifier = LogisticRegression(penalty='l2',solver='liblinear',
                                            max_iter=100,random_state=10,**(self.hparams_model))
        
        elif self.method=='RF':
            classifier = RandomForestClassifier(random_state=10,**self.hparams_model)
            
        return classifier
    
    
    def run_finalise(self):
        '''for finalising (training+testing) model after model_selection'''
        print('\n\n**Retrieved best model and hyper-parameters from model selection and started finalizing..')    
        
        features = self.get_features(self.dataset)
        labels = self.get_labels(self.dataset)
        
        classifier = self.build_classifier()
        
        print('\nStarted training of '+self.method+' on FULL dataset'+self.data_name+'\n')
        
        classifier.fit(features,labels)
        
        print('\nFinished training of '+self.method+' on FULL dataset\n')
        
        print('\nMaking predictions with the best model..\n')
        predictions = classifier.predict(features)
        #train_svm_probs = float64 array, (shape=#train_samples x 1), col2='probab to be BPA' (label=1='the greater label/positive class'), col1 = '..to be nonBPA'
        probs = classifier.predict_proba(features)[:, 1]
        
        indic = list(np.arange(len(self.dataset)))
        save_predictions_classifier(self,'finalised',indic,predictions,
                                    probs,self.classif_method_path)
        
        print('\nEvaluating basic metrics on model performance\n')
        test_auc = self.visualise_evaluate(labels,predictions, probs,self.classif_method_path)

        #pickle.dump(classifier,open(self.classif_method_path+'\\best_model.pkl','wb'))
        
        self.plot_feature_importance(classifier)
        
    
        self.best_run_details = {'preProcessing_1':self.preProcessing_1,
                                 'preProcessing_2':self.preProcessing_2,
                                 'data_name':self.data_name,
                                 'split_method':self.split_method,
                                 'outer_splits':self.outer_splits,
                                 'inner_splits':self.inner_splits,
                                 'method':self.method,
                                 'test_auc':test_auc,
                                 'best_hparams':self.hparams_model}
    
        
    
    def visualise_evaluate(self,labels,predictions, probs,fold_path):
        #probs, train_probs : array size=(samples x 1),, contains probability that each prot is BPA (the positive class), as returned by model
        #ref: https://github.com/WillKoehrsen/Machine-Learning-Projects/blob/master/Random%20Forest%20Tutorial.ipynb
        """get stats and ROC-AUC on training and testing data
        suitable ONLY for BINARY classification and balanced classes!"""
        
        baseline = {}
        
        baseline['accuracy']=0.5
        baseline['roc_auc'] = 0.5
        
        results = {}
        
        #y_score=y_probs=  probability estimates of the positive class
        results['roc_auc'] = roc_auc_score(labels, y_score=probs)
        results['accuracy']=accuracy_score(labels, predictions)
        
        train_results = {}
        
        for metric in ['accuracy', 'roc_auc']:
            print(f'{metric.capitalize()}, Baseline: {round(baseline[metric], 2)} Test: {round(results[metric], 2)}')
        
        # Calculate false positive rates and true positive rates, AUC
        base_fpr, base_tpr, _ = roc_curve(labels, [1 for _ in range(len(labels))])
        model_fpr, model_tpr, test_thresholds = roc_curve(labels, probs)
        ## dont calc AUC like below, cause it's more general and not the specific way you should do it for the AUC of an ROC curve
        #model_auc = auc(model_fpr, model_tpr)
    
        plt.figure(figsize = (8, 6))
        plt.rcParams['font.size'] = 16
        
        # Plot both curves
        plt.plot(base_fpr, base_tpr, 'k', label = 'baseline-0.5 AUC')
        plt.plot(model_fpr, model_tpr, 'r', label = 'model-testing')
        plt.legend(loc='lower right');
        plt.xlabel('False Positive Rate'); 
        plt.ylabel('True Positive Rate'); 
        plt.title(r'SVM ROC Curves  (AUC = %0.2f)' % (results['roc_auc'])); 
        plt.tight_layout()
        
        save_path = fold_path+'/AUC=%0.2f'%(results['roc_auc'])+ '.png'
        
        plt.close('all')
    
        
        return results['roc_auc']
        

    def plot_feature_importance(self,classifier):
        #plot feature importance and save it
        if self.method=='XGBoost':
            #TODO: make chart bigger, have to pass ax object here ..
            xgb.plot_importance(classifier)
            #plt.savefig(self.classif_method_path+'\\feature_importance.png')
        elif self.method=='RF':
            fi_model = pd.DataFrame({'feature': self.get_features(self.dataset),
                   'importance': classifier.feature_importances_}).\
                    sort_values('importance', ascending = False)
            #fi_model.to_csv(self.classif_method_path+'\\feature_importance.csv')

    
    def eval_outer_cv(self,outer_runs_details):
        #compute avg of AUC scores and get best perfomring mmodel hparams
        test_auc_scores = (outer_runs_details['test_auc'])
        avg_test_auc = sum(test_auc_scores)/len(test_auc_scores)
        scores_std = np.std(test_auc_scores)
        #return best_params of best perfomring model on outer CV
        best_score_index = test_auc_scores.index((max(test_auc_scores)))
        best_hparams = outer_runs_details['best_inner_hparams'][best_score_index]

        (pd.DataFrame(outer_runs_details)).to_csv(self.classif_method_path+'\\runs_details.csv') 

        return avg_test_auc,best_hparams,scores_std
        



def load_top_10_features_data_AH():
    '''utility fn for returning df with all BPAD200 proteins with their accession, sequence, 
    #top 10 features used by AH et al 2017 in top perfoming SVM (0.787 AUC) and class-labels'''
    '''
    @index_col: `None` means that a standard index col. will be used string 
    If string is given, then it indicates the column to select as index from loaded ones here
    '''
    proteins_filename = 'BPAD200_Balanced_Not_Scaled_Additional_Features_Protein_Sequences_Added - Copy - Copy.csv'
    top10_feats=['LipoP_Signal_Avg_Length','YinOYang-T-Count','NetPhosK-S-Count','LipoP_SPI_Avg_Length','NetMhcPan-B-AvgRank','TargetP-SecretFlag','YinOYang-AvgDiff1','MBAAgl7_CorCount','PickPocket-Avg','PropFurin-Count_Score']
    cols1 = ['Accession','Sequence','Length']
    cols1.extend(top10_feats)
    cols1.extend(['Label'])

    prot_data = pd.read_csv(proteins_filename,
                           names=cols1,index_col=None)
                           

    if len(prot_data)>400:
        prot_data= prot_data[1::2]
        prot_data.reset_index(drop=True,inplace=True)

    return prot_data        



def concat_dfs_list(dfs_list):
    '''utility fn for concatenating dfs in a list efficiently, handling indexing of new df'''
    #TODO: currently only works for creating dataframe with standard index, ie: not one were a different
    #col. such as 'accession' will be used for index (or was used for index)
    
    dfs_list = [df.reset_index(inplace=False) for df in dfs_list]
    dfs_list = [df.drop('index', axis=1,inplace=False) for df in dfs_list if 'index' in list(df.columns)]
    df= pd.concat(dfs_list,axis=1)
    
    return df


def get_col_name_before_label_col(data):
    '''for large datasets, prefer a functional one-liner to this fn'''
    cols = list(data.columns)
    label_ind = cols.index('Label')
    col_before_label = cols[label_ind-1]
    
    return col_before_label
    

In [ ]:
'''UTILITY FUNCTIONS'''

def make_dir(paths):
    '''creates directory path (or multiple directories if needed) if they 
    #don't exist and returns the path created
    If dir. exists, then nth is done, just path is returned!
    '''

    if isinstance(paths,list):
            
        if len(paths)>2:
            path = paths[0]
            for i in range(1,len(paths)):
                path = os.path.join(path,paths[i])
            if os.path.exists(path) == False :
                os.makedirs(path)
                
        elif len(paths)==2:
            path = os.path.join(paths[0],paths[1])
            if os.path.exists(path) == False :
                os.makedirs(path)
            
        elif len(paths)==1:
            path = paths[0]
            if os.path.exists(path) == False :
                os.mkdir(path= path)
                
        else:
            raise ValueError('Length of list of paths has to be larger than 0')


    elif isinstance(paths,str):
        path = paths
        if os.path.exists(path) == False :
            os.mkdir(path= path)
    
    else:
        raise TypeError('Argument @paths must be list of strings or a string, indicating the dir.path')
    #all paths don't have '/' in the end!
    return path



In [ ]:
'''
Class for performing L.O.Bacteria.O.C.V. on best model & dataset combination, identified during model selection phase
As of 2020 paper, best model: LR-PCA 11 PCs-standardized-
    #using Classifier model class we do hparam tuning for each inner fold again for LOBOV
'''

class Lobov_model():
    def __init__(self,base_path,model_dataset_path,
                 split_method, outer_cv_splits,inner_cv_splits,classif_method,hparams):
        
        self.base_path=base_path
        self.model_dataset_path = model_dataset_path
        
        self.entry={}
        #self.hparams_model={'C': 0.01}
        
        self.hparams_models = hparams
        self.load_data()
        
        self.inner_cv_splits = inner_cv_splits
        self.outer_cv_splits = outer_cv_splits
        self.classif_method= classif_method
        self.split_method = split_method
        
        '''################CLASSIFICATION#########################################'''
        #TODO: sort out duplication with main_model_selection.py
        preProcessing_1 = getattr(self.entry,'preProcessing_1')
        preProcessing_2 = getattr(self.entry,'preProcessing_2')
        data_name = getattr(self.entry,'name')
        dataset = getattr(self.entry,'dataset')
            
        dataset_path=make_dir([base_path,preProcessing_1,preProcessing_2,data_name]) 
        
        Classifier_model_ = Classifier_model(self.classif_method,data_name,dataset,
                                             dataset_path,preProcessing_1,
                                             preProcessing_2,self.split_method,
                                 self.outer_cv_splits,self.inner_cv_splits,
                                 self.hparams_models)
        lobov_data_dir = make_dir(['tut-3-lobov','bpad200_lobov_splits_indic'])
        Classifier_model_.outer_cv(lobov_data_dir)

        #no need to save results of runs again, outside of classif_method_path
        
    def load_data(self):
        prot_cols = 'infer'
        prot_ind=[0]
        fetch = Load_data_model_selection(prot_cols=prot_cols,
                                      prot_ind=prot_ind)
        fetch.populate_data_df(self.model_dataset_path)
        
        data_df = fetch.get_all_data()
        
        #lobocv classify
        #all_data_df has only 1 entry
        for entry in data_df.itertuples(index=False,name='Pandas'):
            self.entry = entry
        
    


In [ ]:
'''MAIN SCRIPT FLOW '''

In [ ]:
'''SETTING PATHS'''
#date_time = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
#tut_path = make_dir(['tut-3-lobov',])#date_time,])

In [ ]:
'''ARGUMENTS'''
'''setting data Path'''

'''CHANGE PATH ACCORDING TO TUT-2'''

best_model_dataset_path = r'.\tut-1-PCA-optimal-model\pre_processing\standardise\PCA\PCs.csv'



In [ ]:
#Specifying parameters for inner-CV that will be done for each species-fold (25 overall)
hparams = {'C': [0.01]}
#specifying that we will do a LOBOV outer validation instead of a standard nested 10-fold CV on BPAD200
split_method = 'lobov'
outer_cv_splits=10
inner_cv_splits=10
classif_method = 'LR'

In [ ]:
lm = Lobov_model(tut_path,best_model_dataset_path,split_method,
                        outer_cv_splits,inner_cv_splits,classif_method,hparams)

In [ ]:
'''plot prototype bar plot LOBOV'''
'''AUC scores not flipped here as in paper'''

res = pd.read_csv(r'.\tut-3-lobov\standardise\PCA\PCs\lobov\LR\runs_details.csv', index_col=[0])

# species
species = res.iloc[:,0].values.tolist()
assert len(species)==25 and isinstance(species,list)
#AUC
auc = res.iloc[:,1].values.tolist()
assert len(auc)==25 and isinstance(auc,list)
#AH_AUC
#received through personal communication with Heinson et. al 2019
auc_ah = [0.94,0.89,1.00,0.83,0.75,0.78,0.44,0.92,0.74,0.94,0.73,1.00,0.61,0.56,0.79,0.50,0.94,0.69,0.52,0.55,0.81,0.69,0.84,1.00,0.58]
assert len(auc)==25

x = np.arange(len(species))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(20,12))
rects1 = ax.bar(x - width/2, auc, width, label='AUC',color=(0, 0.4, 0, 1))
rects2 = ax.bar(x + width/2, auc_ah, width, label='AUC_AH',color=(0, 0, 0, 1))

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('AUC Score')
ax.set_title('LOBOV AUC')
ax.set_xticks(x)
ax.set_xticklabels(species,rotation=45)
ax.legend()


fig.tight_layout()

plt.show()

fig.savefig(tut_path+'\\LOBOV_bar_plot.png')



def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

The LOBOV plot in the paper was build using Microsoft Excel and the data in the runs_details.csv file in the "\standardise\PCA\PCs_of_pca_with_11PCs\lobov\LR" folder in the directory where this jupyter notebook is placed.

The columns for the bacteria species, the LOBOV scores computed in this paper, the LOBOV scores from Heinson et. al 2019 were used from the file, as well as the size of each species dataset (each testing dataset) were used from the runs_details.csv file mentioned above. The latter was calculated during the production and formatting of the figure using Microsoft Excel. Also, as explained in the manuscript, any AUC scores below 0.5 were flipped on the LOBOV plot presented on the manuscript.